# Notebook requêtant wikidata en SPARQL

Imports

In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster

### 1) Récupération de tous les rois de France

In [4]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel (GROUP_CONCAT(DISTINCT ?spouseLabel;separator=", ") AS ?spouses) (GROUP_CONCAT(DISTINCT ?mannerOfDeathLabel;separator=", ") AS ?mannersOfDeath) 
?dateOfBirth ?placeOfBirthLabel ?dateOfDeath ?placeOfDeathLabel ?placeOfBurialLabel ?fatherLabel ?motherLabel ?conflictLabel ?startTime ?endTime

WHERE
{
{?item wdt:P31 wd:Q5 .} 
{?item wdt:P39 wd:Q18384454 .} UNION {?item wdt:P39 wd:Q22923081} UNION {?item wdt:P39 wd:Q3439798}
{?item wdt:P569 ?dateOfBirth} UNION {MINUS {?item wdt:P569 ?dateOfBirth}}
{?item wdt:P19 ?placeOfBirth} UNION {MINUS {?item wdt:P19 ?placeOfBirth}}
{?item wdt:P570 ?dateOfDeath} UNION {MINUS {?item wdt:P570 ?dateOfDeath}}
{?item wdt:P20 ?placeOfDeath} UNION {MINUS {?item wdt:P20 ?placeOfDeath}}
{?item wdt:P509 ?mannerOfDeath} UNION {MINUS {?item wdt:P509 ?mannerOfDeath}}
{?item wdt:P119 ?placeOfBurial} UNION {MINUS {?item wdt:P119 ?placeOfBurial}}
{?item wdt:P22 ?father} UNION {MINUS {?item wdt:P22 ?father}}
{?item wdt:P25 ?mother} UNION {MINUS {?item wdt:P25 ?mother}}
{?item wdt:P26 ?spouse} UNION {MINUS {?item wdt:P26 ?spouse}}
{?item wdt:P607 ?conflict} UNION {MINUS {?item wdt:P607 ?conflict}}
?item p:P39 ?positionHeld.
{?positionHeld pq:P580 ?startTime.} UNION {MINUS {?positionHeld pq:P580 ?startTime.}}
{?positionHeld pq:P582 ?endTime.} UNION {MINUS {?positionHeld pq:P582 ?endTime.}}
  
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr" .
                         ?item rdfs:label ?itemLabel .
                         ?placeOfBirth rdfs:label ?placeOfBirthLabel .
                         ?placeOfDeath rdfs:label ?placeOfDeathLabel .
                         ?mannerOfDeath rdfs:label ?mannerOfDeathLabel .
                         ?placeOfBurial rdfs:label ?placeOfBurialLabel .
                         ?father rdfs:label ?fatherLabel .
                         ?mother rdfs:label ?motherLabel .
                         ?spouse rdfs:label ?spouseLabel .}
}GROUP BY ?item ?itemLabel ?dateOfBirth ?placeOfBirthLabel ?dateOfDeath ?placeOfDeathLabel ?placeOfBurialLabel ?fatherLabel ?motherLabel ?conflictLabel ?startTime ?endTime

""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [5]:
results

{'head': {'vars': ['item',
   'itemLabel',
   'spouses',
   'mannersOfDeath',
   'dateOfBirth',
   'placeOfBirthLabel',
   'dateOfDeath',
   'placeOfDeathLabel',
   'placeOfBurialLabel',
   'fatherLabel',
   'motherLabel',
   'conflictLabel',
   'startTime',
   'endTime']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q134259'},
    'itemLabel': {'xml:lang': 'fr',
     'type': 'literal',
     'value': 'Louis VII de France'},
    'dateOfBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
     'type': 'literal',
     'value': '1120-01-01T00:00:00Z'},
    'placeOfBirthLabel': {'xml:lang': 'fr',
     'type': 'literal',
     'value': 'Paris'},
    'dateOfDeath': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
     'type': 'literal',
     'value': '1180-09-25T00:00:00Z'},
    'placeOfDeathLabel': {'xml:lang': 'fr',
     'type': 'literal',
     'value': 'Paris'},
    'placeOfBurialLabel': {'xml:lang': 'fr',
     'type

In [6]:
rois = []
for r in results['results']['bindings']:
    roi = {}
    if("item" in r):
        roi["wikiID"] = r["item"]["value"].split("http://www.wikidata.org/entity/")[1]
    if("itemLabel" in r):
        roi['nom'] = r['itemLabel']['value']
    if("dateOfBirth" in r):
        roi['dateOfBirth'] = r['dateOfBirth']['value']
    if("placeOfBirthLabel" in r):
        roi['placeOfBirthLabel'] = r['placeOfBirthLabel']['value']
    if("dateOfDeath" in r):
        roi['dateOfDeath'] = r['dateOfDeath']['value']
    if("placeOfDeathLabel" in r):
        roi['placeOfDeathLabel'] = r['placeOfDeathLabel']['value']
    if("mannersOfDeath" in r):
        roi['mannersOfDeath'] = r['mannersOfDeath']['value']
    if("placeOfBurialLabel" in r):
        roi['placeOfBurialLabel'] = r['placeOfBurialLabel']['value']
    if("fatherLabel" in r):
        roi['fatherLabel'] = r['fatherLabel']['value']
    if("motherLabel" in r):
        roi['motherLabel'] = r['motherLabel']['value']
    if("spouses" in r):
        roi['spouses'] = r['spouses']['value']
    if("conflictLabel" in r):
        roi['conflictLabel'] = r['conflictLabel']['value']
    if("startTime" in r):
        roi['startTime'] = r['startTime']['value']
    if("endTime" in r):
        roi['endTime'] = r['endTime']['value']
    rois.append(roi)

In [37]:
df = pd.DataFrame(rois)
df.count()

wikiID                169
nom                   169
dateOfBirth           166
placeOfBirthLabel     148
dateOfDeath           165
placeOfDeathLabel     154
mannersOfDeath        169
placeOfBurialLabel    114
fatherLabel           153
motherLabel           135
spouses               169
startTime              81
endTime                81
dtype: int64

In [38]:
df = df.drop_duplicates()

In [39]:
df.count()

wikiID                169
nom                   169
dateOfBirth           166
placeOfBirthLabel     148
dateOfDeath           165
placeOfDeathLabel     154
mannersOfDeath        169
placeOfBurialLabel    114
fatherLabel           153
motherLabel           135
spouses               169
startTime              81
endTime                81
dtype: int64

In [40]:
pd.options.display.max_rows = 999

In [41]:
df = df.drop_duplicates(subset=["nom"])

In [42]:
df.count()

wikiID                120
nom                   120
dateOfBirth           117
placeOfBirthLabel     102
dateOfDeath           116
placeOfDeathLabel     107
mannersOfDeath        120
placeOfBurialLabel     69
fatherLabel           106
motherLabel            89
spouses               120
startTime              50
endTime                50
dtype: int64

In [43]:
df.sort_values(by=['startTime'])

,wikiID,nom,dateOfBirth,placeOfBirthLabel,dateOfDeath,placeOfDeathLabel,mannersOfDeath,placeOfBurialLabel,fatherLabel,motherLabel,spouses,startTime,endTime
34,Q982667,Mallobaud,0400-01-01T00:00:00Z,NaN,0400-01-01T00:00:00Z,NaN,,NaN,NaN,NaN,,0378-01-01T00:00:00Z,0378-01-01T00:00:00Z
33,Q371744,Marcomir,0400-01-01T00:00:00Z,NaN,0392-01-01T00:00:00Z,NaN,,NaN,NaN,NaN,,0380-01-01T00:00:00Z,0400-01-01T00:00:00Z
30,Q82339,Clovis Ier,0466-01-01T00:00:00Z,Tournai,0511-11-29T00:00:00Z,Paris,,abbaye Sainte-Geneviève de Paris,Childéric Ier,Basine de Thuringe,"princesse franque, Clotilde",0481-01-01T00:00:00Z,0511-11-29T00:00:00Z
50,Q309909,Childéric II,0655-01-01T00:00:00Z,t1791293328,0675-01-01T00:00:00Z,Chelles,,abbaye de Saint-Germain-des-Prés,Clovis II,Bathilde,Bilichilde,0673-01-01T00:00:00Z,0675-01-01T00:00:00Z
57,Q295175,Thierry III,0657-01-01T00:00:00Z,t1795134883,0691-01-01T00:00:00Z,t1795134884,,Arras,Clovis II,Bathilde,Clotilde dite Doda,0675-01-01T00:00:00Z,0679-01-01T00:00:00Z
52,Q316874,Dagobert II,0652-01-01T00:00:00Z,t1792225373,0679-12-26T00:00:00Z,Mouzay,,NaN,Sigebert III,Chimnechilde,,0676-01-01T00:00:00Z,0679-12-26T00:00:00Z
32,Q309934,Clovis IV,0677-01-01T00:00:00Z,t1794603784,0695-01-01T00:00:00Z,t1794603788,,NaN,Thierry III,Clotilde dite Doda,,0691-01-01T00:00:00Z,0695-01-01T00:00:00Z
2,Q109223,Childebert IV,0678-01-01T00:00:00Z,t1789344646,0711-04-18T00:00:00Z,t1789344648,,Choisy-au-Bac,Thierry III,Clotilde dite Doda,,0695-01-01T00:00:00Z,0711-01-01T00:00:00Z
21,Q109248,Dagobert III,0698-01-01T00:00:00Z,t1789345170,0715-01-01T00:00:00Z,t1789345172,,NaN,Childebert IV,NaN,,0711-01-01T00:00:00Z,0715-01-01T00:00:00Z
53,Q316908,Clotaire IV,0685-01-01T00:00:00Z,t1794603790,0719-01-01T00:00:00Z,t1794603789,,NaN,Childebert IV,NaN,,0717-01-01T00:00:00Z,0719-01-01T00:00:00Z


In [44]:
df2 = df[df['startTime'].notna() & df['endTime'].notna()]

In [45]:
df2 = df2[df2['dateOfBirth'].notna() & df2['dateOfDeath'].notna()]

In [46]:
df2.shape

(50, 13)

In [47]:
from datetime import datetime

In [48]:
df2['startYear'] = df2['startTime'].map(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").year)
df2['endYear'] = df2['endTime'].map(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").year)


In [49]:
df2

,wikiID,nom,dateOfBirth,placeOfBirthLabel,dateOfDeath,placeOfDeathLabel,mannersOfDeath,placeOfBurialLabel,fatherLabel,motherLabel,spouses,startTime,endTime,startYear,endYear
0,Q134259,Louis VII de France,1120-01-01T00:00:00Z,Paris,1180-09-25T00:00:00Z,Paris,,basilique Saint-Denis,Louis VI de France,Adélaïde de Savoie,"Constance de Castille, Adèle de Champagne, Ali...",1137-08-08T00:00:00Z,1180-09-25T00:00:00Z,1137,1180
1,Q165284,Louis VIII de France,1187-09-12T00:00:00Z,Paris,1226-11-15T00:00:00Z,Montpensier,dysentérie,basilique Saint-Denis,Philippe II Auguste,Isabelle de Hainaut,Blanche de Castille,1223-07-21T00:00:00Z,1226-11-15T00:00:00Z,1223,1226
2,Q109223,Childebert IV,0678-01-01T00:00:00Z,t1789344646,0711-04-18T00:00:00Z,t1789344648,,Choisy-au-Bac,Thierry III,Clotilde dite Doda,,0695-01-01T00:00:00Z,0711-01-01T00:00:00Z,695,711
4,Q133372,Charles VII de France,1403-03-03T00:00:00Z,Paris,1461-07-31T00:00:00Z,Mehun-sur-Yèvre,,basilique Saint-Denis,Charles VI,Isabeau de Bavière,Marie d'Anjou,1422-11-08T00:00:00Z,1461-08-09T00:00:00Z,1422,1461
5,Q108331,Thierry IV,0712-01-01T00:00:00Z,t1789308418,0737-01-01T00:00:00Z,t1789308419,,basilique Saint-Denis,Dagobert III,NaN,,0721-01-01T00:00:00Z,0737-01-01T00:00:00Z,721,737
6,Q159575,Hugues Capet,0940-01-01T00:00:00Z,Dourdan,0996-10-29T00:00:00Z,Les Juifs,,basilique Saint-Denis,Hugues le Grand,Hedwige de Saxe,Adélaïde d'Aquitaine,0987-06-08T00:00:00Z,0996-10-29T00:00:00Z,987,996
7,Q165883,Louis VI de France,1081-12-07T00:00:00Z,Paris,1137-08-08T00:00:00Z,château de la Douye,,basilique Saint-Denis,Philippe Ier de France,Berthe de Hollande,"Lucienne de Rochefort, Adélaïde de Savoie",1108-08-06T00:00:00Z,1137-08-08T00:00:00Z,1108,1137
8,Q109240,Chilpéric II,0673-01-01T00:00:00Z,t1789344734,0721-02-17T00:00:00Z,Nyon,,Noyon,Childéric II,Bilichilde,,0719-01-01T00:00:00Z,0721-01-01T00:00:00Z,719,721
12,Q132545,Henri II,1519-04-10T00:00:00Z,Saint-Germain-en-Laye,1559-07-20T00:00:00Z,hôtel des Tournelles,,basilique Saint-Denis,François Ier,Claude de France,Catherine de Médicis,1547-04-10T00:00:00Z,1559-07-20T00:00:00Z,1547,1559
13,Q134452,Charles VIII,1470-07-09T00:00:00Z,Amboise,1498-04-16T00:00:00Z,Amboise,accident vasculaire cérébral,basilique Saint-Denis,Louis XI,Charlotte de Savoie,Anne de Bretagne,1483-09-17T00:00:00Z,1498-04-25T00:00:00Z,1483,1498


In [50]:
df2 = df2[df2["startYear"] > 986]

In [51]:
df2['birthYear'] = df2['dateOfBirth'].map(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").year)
df2['deathYear'] = df2['dateOfDeath'].map(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").year)

/home/geof/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/geof/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
df2.dtypes

wikiID                object
nom                   object
dateOfBirth           object
placeOfBirthLabel     object
dateOfDeath           object
placeOfDeathLabel     object
mannersOfDeath        object
placeOfBurialLabel    object
fatherLabel           object
motherLabel           object
spouses               object
startTime             object
endTime               object
startYear              int64
endYear                int64
birthYear              int64
deathYear              int64
dtype: object

In [53]:
def which_king(year):
    df_king = df2[df2["startYear"] <= year]
    df_king = df_king[df_king["endYear"] > year]
    return df_king

In [54]:
king = which_king(1609)
king

,wikiID,nom,dateOfBirth,placeOfBirthLabel,dateOfDeath,placeOfDeathLabel,mannersOfDeath,placeOfBurialLabel,fatherLabel,motherLabel,spouses,startTime,endTime,startYear,endYear,birthYear,deathYear
55,Q936976,Henri IV,1553-12-23T00:00:00Z,château de Pau,1610-05-14T00:00:00Z,Paris,blessure par arme blanche,basilique Saint-Denis,Antoine de Bourbon,Jeanne d'Albret,"Marguerite de France, Marie de Médicis",1572-06-19T00:00:00Z,1610-05-14T00:00:00Z,1572,1610,1553,1610


### 2) Save in csv

In [55]:
df2.to_csv("rois-france-avec-dates.csv", index=False)

In [56]:
df2

,wikiID,nom,dateOfBirth,placeOfBirthLabel,dateOfDeath,placeOfDeathLabel,mannersOfDeath,placeOfBurialLabel,fatherLabel,motherLabel,spouses,startTime,endTime,startYear,endYear,birthYear,deathYear
0,Q134259,Louis VII de France,1120-01-01T00:00:00Z,Paris,1180-09-25T00:00:00Z,Paris,,basilique Saint-Denis,Louis VI de France,Adélaïde de Savoie,"Constance de Castille, Adèle de Champagne, Ali...",1137-08-08T00:00:00Z,1180-09-25T00:00:00Z,1137,1180,1120,1180
1,Q165284,Louis VIII de France,1187-09-12T00:00:00Z,Paris,1226-11-15T00:00:00Z,Montpensier,dysentérie,basilique Saint-Denis,Philippe II Auguste,Isabelle de Hainaut,Blanche de Castille,1223-07-21T00:00:00Z,1226-11-15T00:00:00Z,1223,1226,1187,1226
4,Q133372,Charles VII de France,1403-03-03T00:00:00Z,Paris,1461-07-31T00:00:00Z,Mehun-sur-Yèvre,,basilique Saint-Denis,Charles VI,Isabeau de Bavière,Marie d'Anjou,1422-11-08T00:00:00Z,1461-08-09T00:00:00Z,1422,1461,1403,1461
6,Q159575,Hugues Capet,0940-01-01T00:00:00Z,Dourdan,0996-10-29T00:00:00Z,Les Juifs,,basilique Saint-Denis,Hugues le Grand,Hedwige de Saxe,Adélaïde d'Aquitaine,0987-06-08T00:00:00Z,0996-10-29T00:00:00Z,987,996,940,996
7,Q165883,Louis VI de France,1081-12-07T00:00:00Z,Paris,1137-08-08T00:00:00Z,château de la Douye,,basilique Saint-Denis,Philippe Ier de France,Berthe de Hollande,"Lucienne de Rochefort, Adélaïde de Savoie",1108-08-06T00:00:00Z,1137-08-08T00:00:00Z,1108,1137,1081,1137
12,Q132545,Henri II,1519-04-10T00:00:00Z,Saint-Germain-en-Laye,1559-07-20T00:00:00Z,hôtel des Tournelles,,basilique Saint-Denis,François Ier,Claude de France,Catherine de Médicis,1547-04-10T00:00:00Z,1559-07-20T00:00:00Z,1547,1559,1519,1559
13,Q134452,Charles VIII,1470-07-09T00:00:00Z,Amboise,1498-04-16T00:00:00Z,Amboise,accident vasculaire cérébral,basilique Saint-Denis,Louis XI,Charlotte de Savoie,Anne de Bretagne,1483-09-17T00:00:00Z,1498-04-25T00:00:00Z,1483,1498,1470,1498
14,Q160349,Charles VI,1368-12-11T00:00:00Z,Paris,1422-10-30T00:00:00Z,Paris,paludisme,basilique Saint-Denis,Charles V de France,Jeanne de Bourbon,Isabeau de Bavière,1380-09-24T00:00:00Z,1422-10-30T00:00:00Z,1380,1422,1368,1422
15,Q178908,Henri Ier de France,1008-05-04T00:00:00Z,Reims,1060-08-04T00:00:00Z,Vitry-aux-Loges,,basilique Saint-Denis,Robert II de France,Constance d'Arles,"Mathilde de Franconie, Anne de Kiev, Mathilde ...",1031-07-16T00:00:00Z,1060-08-10T00:00:00Z,1031,1060,1008,1060
16,Q167782,Charles V de France,1338-01-29T00:00:00Z,Vincennes,1380-09-24T00:00:00Z,Paris,maladie infectieuse,basilique Saint-Denis,Jean II de France,Bonne de Luxembourg,Jeanne de Bourbon,1364-04-16T00:00:00Z,1380-09-24T00:00:00Z,1364,1380,1338,1380


In [57]:
df2.columns

Index(['wikiID', 'nom', 'dateOfBirth', 'placeOfBirthLabel', 'dateOfDeath',
       'placeOfDeathLabel', 'mannersOfDeath', 'placeOfBurialLabel',
       'fatherLabel', 'motherLabel', 'spouses', 'startTime', 'endTime',
       'startYear', 'endYear', 'birthYear', 'deathYear'],
      dtype='object')

## Récup image si possible

In [58]:
from wikidata.client import Client
client = Client()  # doctest: +SKIP

In [60]:
updKings = []
for index, row in df2.iterrows():
    print(row["nom"])
    updKing = row

    entity = client.get(row["wikiID"], load=True)
    try:
        image_prop = client.get('P18')
        image = entity[image_prop]
        updKing["urlImage"] = image.image_url
    except:
        updKing["urlImage"] = ""
    updKings.append(updKing)
df_upd = pd.DataFrame(updKings)
df_upd

Louis VII de France
Louis VIII de France
Charles VII de France
Hugues Capet
Louis VI de France
Henri II
Charles VIII
Charles VI
Henri Ier de France
Charles V de France
Philippe VI de France
Charles IV de France
Robert II de France
Charles IX
Philippe IV de France
François II
Louis XIII
Henri IV
François Ier
Jean II de France
Philippe Ier de France
Philippe III de France
Philippe II Auguste
Saint Louis
Henri III
Louis XII
Louis XI
Louis XVI
Louis XIV
Charles X
Philippe V de France
Louis XV
Louis XVIII


,wikiID,nom,dateOfBirth,placeOfBirthLabel,dateOfDeath,placeOfDeathLabel,mannersOfDeath,placeOfBurialLabel,fatherLabel,motherLabel,spouses,startTime,endTime,startYear,endYear,birthYear,deathYear,urlImage
0,Q134259,Louis VII de France,1120-01-01T00:00:00Z,Paris,1180-09-25T00:00:00Z,Paris,,basilique Saint-Denis,Louis VI de France,Adélaïde de Savoie,"Constance de Castille, Adèle de Champagne, Ali...",1137-08-08T00:00:00Z,1180-09-25T00:00:00Z,1137,1180,1120,1180,https://upload.wikimedia.org/wikipedia/commons...
1,Q165284,Louis VIII de France,1187-09-12T00:00:00Z,Paris,1226-11-15T00:00:00Z,Montpensier,dysentérie,basilique Saint-Denis,Philippe II Auguste,Isabelle de Hainaut,Blanche de Castille,1223-07-21T00:00:00Z,1226-11-15T00:00:00Z,1223,1226,1187,1226,https://upload.wikimedia.org/wikipedia/commons...
4,Q133372,Charles VII de France,1403-03-03T00:00:00Z,Paris,1461-07-31T00:00:00Z,Mehun-sur-Yèvre,,basilique Saint-Denis,Charles VI,Isabeau de Bavière,Marie d'Anjou,1422-11-08T00:00:00Z,1461-08-09T00:00:00Z,1422,1461,1403,1461,https://upload.wikimedia.org/wikipedia/commons...
6,Q159575,Hugues Capet,0940-01-01T00:00:00Z,Dourdan,0996-10-29T00:00:00Z,Les Juifs,,basilique Saint-Denis,Hugues le Grand,Hedwige de Saxe,Adélaïde d'Aquitaine,0987-06-08T00:00:00Z,0996-10-29T00:00:00Z,987,996,940,996,https://upload.wikimedia.org/wikipedia/commons...
7,Q165883,Louis VI de France,1081-12-07T00:00:00Z,Paris,1137-08-08T00:00:00Z,château de la Douye,,basilique Saint-Denis,Philippe Ier de France,Berthe de Hollande,"Lucienne de Rochefort, Adélaïde de Savoie",1108-08-06T00:00:00Z,1137-08-08T00:00:00Z,1108,1137,1081,1137,https://upload.wikimedia.org/wikipedia/commons...
12,Q132545,Henri II,1519-04-10T00:00:00Z,Saint-Germain-en-Laye,1559-07-20T00:00:00Z,hôtel des Tournelles,,basilique Saint-Denis,François Ier,Claude de France,Catherine de Médicis,1547-04-10T00:00:00Z,1559-07-20T00:00:00Z,1547,1559,1519,1559,https://upload.wikimedia.org/wikipedia/commons...
13,Q134452,Charles VIII,1470-07-09T00:00:00Z,Amboise,1498-04-16T00:00:00Z,Amboise,accident vasculaire cérébral,basilique Saint-Denis,Louis XI,Charlotte de Savoie,Anne de Bretagne,1483-09-17T00:00:00Z,1498-04-25T00:00:00Z,1483,1498,1470,1498,https://upload.wikimedia.org/wikipedia/commons...
14,Q160349,Charles VI,1368-12-11T00:00:00Z,Paris,1422-10-30T00:00:00Z,Paris,paludisme,basilique Saint-Denis,Charles V de France,Jeanne de Bourbon,Isabeau de Bavière,1380-09-24T00:00:00Z,1422-10-30T00:00:00Z,1380,1422,1368,1422,https://upload.wikimedia.org/wikipedia/commons...
15,Q178908,Henri Ier de France,1008-05-04T00:00:00Z,Reims,1060-08-04T00:00:00Z,Vitry-aux-Loges,,basilique Saint-Denis,Robert II de France,Constance d'Arles,"Mathilde de Franconie, Anne de Kiev, Mathilde ...",1031-07-16T00:00:00Z,1060-08-10T00:00:00Z,1031,1060,1008,1060,https://upload.wikimedia.org/wikipedia/commons...
16,Q167782,Charles V de France,1338-01-29T00:00:00Z,Vincennes,1380-09-24T00:00:00Z,Paris,maladie infectieuse,basilique Saint-Denis,Jean II de France,Bonne de Luxembourg,Jeanne de Bourbon,1364-04-16T00:00:00Z,1380-09-24T00:00:00Z,1364,1380,1338,1380,https://upload.wikimedia.org/wikipedia/commons...


In [61]:
df_upd.to_csv("../csv/rois-france-avec-dates.csv", index=False)